---
title: From PyTorch to tinygrad using ONNX
description: Training a PyTorch model and converting it to ONNX and then to tinygrad.
author: Kelly To
date: '2024-09-12'
categories: [PyTorch, tinygrad, MNIST, ONNX]
---

-----

This post demonstrates how to use ONNX to get tinygrad to run a PyTorch model. This is an alternative to writing a whole tinygrad version of the PyTorch model you want to port, like we did in the post [From PyTorch to tinygrad](/blog/posts/1_from_pytorch_to_tinygrad.qmd).

The PyTorch model I'm exporting is from the PyTorch file I used in that post, 'pytorch_MNIST_start.ipynb'. The tinygrad code is in a separate file that I made called 'tinygrad_MNIST_onnx.ipynb'. Below is the relevant code from the two files.

<br>

#### PyTorch
## Export PyTorch model as an ONNX model
###### In my 'pytorch_MNIST_start.ipynb' file, I added the following code to the end of the notebook to export the model as an ONNX model in a file called 'mnist_model.onnx'.

In [ ]:
import torch.onnx

dummy_input = torch.randn(1, 1, 28, 28)
torch.onnx.export(model, 
                  dummy_input, 
                  "mnist_model.onnx")

<br>

#### tinygrad
## Load ONNX model in tinygrad
###### This loads the ONNX model in tinygrad and creates a callable object 'run_onnx' that can execute the model.

In [39]:
import onnx
from extra.onnx import get_run_onnx

model = onnx.load("mnist_model.onnx")
run_onnx = get_run_onnx(model)

## Count parameters
###### I tweaked the 'count_parameters' function from the previous blog post, [Pretty Table for Parameters](/blog/posts/3_pretty_table_for_parameters.qmd), to work with ONNX models. Looking at the table, we can confirm that the ONNX model has the same number of parameters as the PyTorch model.

In [41]:
from prettytable import PrettyTable
import numpy as np

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    # ONNX uses model.graph.initializer to iterate through the parameters (nodes)
    for node in model.graph.initializer:
        # ONNX uses np.prod() to calculate parameter count by multiplying the dimensions (node.dims = parameter shape)
        num_params = np.prod(node.dims)
        table.add_row([node.name, num_params])
        total_params += num_params
    print(table)
    print(f"Total Trainable Params: {total_params}\n")
    return total_params

count_parameters(model)

+-----------+------------+
|  Modules  | Parameters |
+-----------+------------+
| l1.weight |    288     |
|  l1.bias  |     32     |
| l2.weight |   18432    |
|  l2.bias  |     64     |
| l3.weight |   16000    |
|  l3.bias  |     10     |
+-----------+------------+
Total Trainable Params: 34826



np.int64(34826)

## Get the MNIST dataset
###### I imported the MNIST dataset to grab an image to use to test the model.

In [ ]:
from tinygrad.nn.datasets import mnist

X_train, Y_train, X_test, Y_test = mnist()
print(X_train.shape, X_train.dtype, Y_train.shape, Y_train.dtype)

## Final probabilities
###### If you refer back to the final tinygrad probabilities in [From PyTorch to tinygrad](/blog/posts/1_from_pytorch_to_tinygrad.qmd), you will see that the values below are exactly the same.

In [44]:
# Select the first test image
test_image = X_test[0:1]

# Run the ONNX model using run_onnx function with the test image as input
# The model expects an input with the key "input.1"
onnx_output = run_onnx({"input.1": test_image})

# Get the output tensor (single vector of 10 values, 1 for each digit class)
output_tensor = list(onnx_output.values())[0]

# Apply softmax and convert to numpy
tinygrad_probs = output_tensor.softmax().numpy()

# Print the resulting probabilities
print("tinygrad probabilities:", tinygrad_probs)

tinygrad probabilities: [[2.6173244e-14 4.2464655e-14 6.4881434e-08 4.5528861e-09 2.0712009e-17
  4.9732746e-11 1.3766536e-21 9.9999988e-01 8.9217121e-13 8.5283941e-10]]
